# Derivation of Classical Multistep Methods

In [ ]:
using Pkg
Pkg.activate(".")
using SymPy

## Lagrange interpolating polynomials
[Wikipedia](https://en.wikipedia.org/wiki/Lagrange_polynomial) has a practical definition of the Lagrange interpolating polynomial.  This is basically a polynomial that when given $n$ points passes through each point and is of order $n-1$.  

Given a set of $k+1$ data points
$$(t_0, x_0),\ldots,(t_j, x_j),\ldots,(t_k, x_k)$$
where no two $t_j$ are the same, the ''interpolation polynomial in the Lagrange form'' is a linear combination
$$L(t) := \sum_{j=0}^{k} x_j \ell_j(t)$$
of Lagrange basis polynomials
$$\ell_j(t) := \prod_{\begin{smallmatrix}0\le m\le k\\ m\neq j\end{smallmatrix}} \frac{t-t_m}{t_j-t_m} = \frac{(t-t_0)}{(t_j-t_0)} \cdots \frac{(t-t_{j-1})}{(t_j-t_{j-1})} \frac{(t-t_{j+1})}{(t_j-t_{j+1})} \cdots \frac{(t-t_k)}{(t_j-t_k)},$$

where $0 \le j\le k$. Note how, given the initial assumption that no two $t_i$ are the same, $t_j - t_m \neq 0$, so this expression is always well-defined. 

Let's define a bunch of variables to use later.

In [ ]:
x1, x2, x3, x4, x5 = symbols("x1, x2, x3, x4, x5")
t1, t2, t3, t4, t5 = symbols("t1, t2, t3, t4, t5")
t = symbols("t")
h = symbols("h", positive=true)
X = [x1,x2,x3,x4,x5]
T = [t1,t2,t3,t4,t5]
;

Using the definition above, we can now define the inner part of the Lagrange interpolating polynomials $l$

In [ ]:
function l(t,j,k)
    l = 1
    for m in 1:k
        if m != j
            l *= ( t - T[m] )/( T[j] - T[m] )
        end        
    end
    return l
end
;

## Adams-Bashforth
We can setup this problem as one of interpolation.  We want to approximate
$$ x_{n+s} = x_{n+s-1} + \int\limits_{t_{n+s-1}}^{ t_{n+s}} f(x,t)\,dt$$
and we can approximate $f(x,t)$ with a Lagrange polynomial from the previous several steps.
$$f(x,t) \approx p(t) =  \sum_{j=0}^{k} f(x_j,t_j) \ell_j(t)$$

The explicit part of Adams-Bashforth is that **only** previous points are considered.  So in the numbering used in this sheet, if we want to know what's happening at $f_3$, then we build an interpolation that involves only data from $f_2$ and $f_1$.

In [ ]:
f1,f2,f3,f4,f5 = symbols("f_1, f_2, f_3, f_4, f_5")
F = [f1,f2,f3,f4,f5]
;

## Example: AB2

In [ ]:
n = 2
P = [ F[i]*l(t, i, n) for i in 1:n ] |> sum

In [ ]:
intP = integrate(P, (t, T[n], T[n+1]) ) |> simplify

Now let's assume that the step size is a constant $h$

In [ ]:
intP_h = intP |> subs( t2=>t1+h, t3=> t1+2*h) |> simplify

Finally we arrive at the expression for the classic AB2 method

In [ ]:
Eq(x3, x2 + intP_h)

## Adams-Moulton
These methods differ from Bashforth in how $p(t)$ is built.  Instead of only using previous values, it involves the unknown current value as well.  This makes AM methods *implicit*.

Compared to the analysis before, the interpolation $P$ is bumped up 1 order, bounds of the integral are adjusted.

### AB3: Two previous states, 
This is the same storage need as before in AB2

In [ ]:
n = 3
P = [ F[i]*l(t, i, n) for i in 1:n ] |> sum

In [ ]:
intP = integrate(P, (t, T[n-1], T[n] ) ) |> simplify

Again, let's assume that the step size is constant $h$

In [ ]:
intP_h = intP |> subs( t2=>t1+h, t3=> t1+2*h ) |> simplify
Eq(x3, x2 + intP_h)